In [ ]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from os import path, environ
from datetime import datetime
from services.zayev.environment.market_simulator import MarketSimulator
from services.zayev.service_layer.forester import Forester
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib.dates import relativedelta
from apps.environment.service_layer.data_transformer import DataTransformer
import pandas as pd
from joblib import load
import pathlib
import pytz

In [ ]:
from copy import deepcopy


nwb_starting_time_step = datetime(year=2023, month=1, day=1)
now_time = pytz.utc.localize(datetime(year=2024, month=1, day=3))

# now_time = pytz.utc.localize(datetime.now())
# nwb_starting_time_step = datetime.now() - relativedelta(days=360)
# nwb_starting_time_step = nwb_starting_time_step.strftime('%Y-%m-%d %H:%M:%S')
# nwb_starting_time_step = datetime.strptime(nwb_starting_time_step, "%Y-%m-%d %H:%M:%S")

nwb_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nwb_env_config = {
    "db_params": nwb_db_params, 
    "max_episode_steps": 1000, 
    "the_current_time_step": nwb_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": False,
    "no_of_stocks": 199,
    "no_of_cmmdts": 14,
    "is_live": True,
    # "temp_data_date": "2024-01-02",
    "is_crypto": True
} 
nwb_dformer = DataTransformer(env_config=nwb_env_config)
nwb_dfs = nwb_dformer.create_input_dataframes()
stck_buffs = nwb_dfs[2]
# stck_buffs = stck_buffs[stck_buffs['captured_at'] >= '2024-01-01']
# nwb_dfs[2] = stck_buffs
# display(nwb_dfs[2])
# pd.set_option('display.max_rows', None)
# display(stck_buffs)
temp_df = stck_buffs[stck_buffs["index"] == 29][['captured_at', 'price_snapshot', 'x_day_ma']]
temp_df = temp_df[temp_df['captured_at'] <= '2023-06-27'].tail(20)
# display(temp_df)
middle_column_sum = temp_df['price_snapshot'].sum()
# print("middle_column_sum", middle_column_sum/20)
nwb_market = MarketSimulator(env_config=nwb_env_config, data_frames=nwb_dfs)
nwb_market.reset()

total_init_potfolio = nwb_market.wallet_state
# print("asd", nwb_market.stock_data)
for initd in range(199):
    data_initd = initd+1
    total_init_potfolio = total_init_potfolio + nwb_market.stock_data[data_initd]["price_snapshot"]*nwb_market.shares_data[data_initd]
print(total_init_potfolio)
print(nwb_market.the_current_time_step - relativedelta(hours=100))

nwb_forest = Forester(market=nwb_market)
total_potfolio = nwb_market.wallet_state
latest_action = None
print(nwb_market.stock_data)
for ind in range(199):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]

initial_total_potfolio = total_potfolio
print(total_potfolio)
print(nwb_market.the_current_time_step, now_time)
changes = []
selected_acts = []
for indx in range(550):
    if indx < 20:
        acts = np.full(199, 15)
        rand_acts = nwb_forest.act_random(acts)
        nwb_market.step(rand_acts)
    else:
        rand_acts = nwb_forest.market.get_mov_avg_actions()
        # rand_acts = nwb_forest.market.get_gradient_acts()
        # print(rand_acts)
        acts_list = []
        iny = 1
        # rand_acts = np.full(199, 15)
        # if nwb_market.is_new_month:
        #     acts = np.full(100, 15)
        #     rand_acts = nwb_forest.act_random(acts)
        # else:
        #     rand_acts = nwb_forest.act_random(rand_acts, use_best_stks=False)
        rand_acts = nwb_forest.act_random(rand_acts, use_best_stks=False)
        for an_act in rand_acts:
            if iny == 159:
                selected_acts.append((nwb_market.the_current_time_step, an_act, nwb_market.curr_signals.get(159, None)))
            acts_list.append((iny, an_act))
            iny=iny+1
        the_time_step = nwb_market.the_current_time_step
        # print("my time", nwb_market.the_current_time_step, now_time, nwb_market.the_current_time_step > now_time)
        print("acts_list", acts_list)
        print("shares data", nwb_market.shares_data)
        nwb_market.step(rand_acts)
        print("abs_change_in_shares", nwb_market.abs_change_in_shares)
        for ashr in nwb_market.abs_change_in_shares:
            if nwb_market.abs_change_in_shares[ashr] !=0:
                # print("changed share", ashr)
                changes.append((the_time_step, acts_list, deepcopy(nwb_market.abs_change_in_shares)))
                break
        if nwb_market.the_current_time_step > now_time :
            latest_action = acts
            break
        print("")
        # if indx>40:
        #     break

# print(changes)
# for (date, action, signal) in selected_acts:
#     signal.pop("prev_signals")
# print(nwb_market.wallet_state)

In [ ]:
print(nwb_market.curr_signals)

In [ ]:
# pd_signals = []
# for (date, action, signal) in selected_acts:
#     # print(nd_acts)
#     # signal.pop("prev_signals")
#     signal["date"] = str(date)
#     signal["action"] = action
#     pd_signals.append(signal)

# signals_df = pd.DataFrame(pd_signals)
# columns_to_place_first = ['date', 'action']

# # Rearrange columns
# signals_df = signals_df[columns_to_place_first + [col for col in signals_df.columns if col not in columns_to_place_first]]
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
my_df = stck_buffs

pivot_df = my_df.pivot(index='captured_at', columns='index', values='price_snapshot_percentage_change')

# If you want to reset the index to have 'captured_at' as a regular column
pivot_df.reset_index(inplace=True)

# Print the resulting DataFrame
# display(pivot_df)

numerical_cols = pivot_df.columns[1:]
# display(numerical_cols)
pivot_df[numerical_cols] = pivot_df[numerical_cols].apply(pd.to_numeric, errors='coerce')
top_20_values = pivot_df[numerical_cols].apply(lambda row: row.nlargest(20).index, axis=1)

for index, row in pivot_df.iterrows():
    pivot_df.loc[index, numerical_cols] = np.where(pivot_df.loc[index, numerical_cols].index.isin(top_20_values.loc[index]), pivot_df.loc[index, numerical_cols], 0)

# Display the resulting DataFrame
display(pivot_df)

In [ ]:
287	2023-10-15 00:00:00+00:00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2.536790	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	8.263383e+03	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2.270220	0.000000	0.000000	0.000000	3.272635	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	3.322560	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2.404039	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	10.751364	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	3.966181	0.000000	0.000000	0.000000	8.961189	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2.699129	9.686099	0.000000	4.107603	0.000000	0.000000	0.000000	0.000000	11.539066	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2.146370	3.424769	2.389597	0.000000	0.000000	0.000000	2.134453	0.000000	0.000000	0.000000	0.000000	2.491302	0.000000	0.000000	0.000000	0.000000	2.244172	0.000000	0.000000	2.773329	0.000000
# 288	2023-10-16 00:00:00+00:00	0.000000	0.000000	0.000000	0.000000	9.401184	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	5.152812	0.000000	0.000000	0.000000	0.000000	7.561014	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000e+00	6.129122	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	5.066374	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	15.349079	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	6.277635	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	6.922776	0.000000	0.000000	0.000000	0.000000	5.599581	0.000000	0.000000	0.000000	0.000000	7.038109	0.000000	0.000000	0.000000	34.594362	10.340332	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	16.189697	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	5.010134	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	12.329175	5.335087	7.180150	0.000000	0.000000	0.000000	0.000000	0.000000	8.515382	0.000000	0.000000	8.194501	0.000000	0.000000	9.791852	0.000000
# 289	2023-10-17 00:00:00+00:00	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	7.013930	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	1.214414	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	4.876709	0.000000	0.000000	0.000000	0.000000e+00	1.839752	0.000000	0.000000	0.000000	0.000000	0.000000	5.920920	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2.315320	1.738408	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	15.369262	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	1.908961	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	1.563501	0.000000	1.183467	0.000000	0.000000	0.000000	1.699158	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	1.189589	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	1.832693	0.000000	0.000000	2.267655	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2.417840	0.000000	0.000000	2.662544	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	3.640587	0.000000	2.476610	0.000000	0.000000	0.000000	0.000000	0.000000	6.371894	0.000000
# 290

In [ ]:
print(nwb_market.shares_data)
total_potfolio = nwb_market.wallet_state
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]
print(total_potfolio)
print(nwb_market.the_current_time_step - relativedelta(hours=100))
final_total_potfolio = total_potfolio
print(nwb_market.total_added)
gain = final_total_potfolio - initial_total_potfolio - nwb_market.total_added
print("gain", gain)

In [ ]:
import numpy as np
# bad year
# 1, -231.04535665378006
# 2, -535.5779944116628
# 3, -141.01900894586396
# Assuming avg_acts is your array and max_val, min_val are constants
# Replace them with your actual values

## good year:
# 1, 605001.6503871948
# 2, 2892705.172230803
# 3, 3545.134111095337


avg_acts = np.array([1, -2, 3, -4, 5])
max_val = 10
min_val = -10

# Separate positive and negative values
positive_values = avg_acts[avg_acts >= 0]
negative_values = avg_acts[avg_acts < 0]

# Calculate the scaled array for positive values
scaled_positive = positive_values * ((max_val - 0) / (positive_values.max() - 0))

# Calculate the scaled array for negative values
scaled_negative = negative_values * ((0 - min_val) / (0 - negative_values.min()))

# Combine the scaled arrays back together
scaled_array = np.empty_like(avg_acts)
scaled_array[avg_acts >= 0] = scaled_positive
scaled_array[avg_acts < 0] = scaled_negative

print(scaled_array)